In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def generateVertices(N, r, z, W, H):
    '''method to generate the vertices for a set of semicircular riblets
    
    N: no. of riblets
    r: semicircle radius
    z: length of system in 3D z direction
    W: riblet subsystem padding width (this is fixed, so need to make sure it's bigger than the semicicle outer block sizes)
    H: system height'''
    
    R = 2.5*r
    W = 4*R
    
    n_bottom = 9 #no. of vertices in the bottom row of the riblet system
    n_middle = 5
    n_top = 5
    
    vertices_bottom = [None]*(n_bottom*N)
    vertices_middle = [None]*(n_middle*N)
    vertices_top = [None]*(n_top*N)
    
    for i in range(N):
        
        #bottom vertices
        vertices_bottom[0+i*n_bottom] = (-W, 0, 0) #LHS
        vertices_bottom[1+i*n_bottom] = (R*np.cos(np.pi), 0, 0) #base of riblet LHS
        vertices_bottom[2+i*n_bottom] = (r*np.cos(np.pi), 0, 0) 
        vertices_bottom[3+i*n_bottom] = (r*np.cos(3*np.pi/4), r*np.sin(3*np.pi/4), 0) 
        vertices_bottom[4+i*n_bottom] = (r*np.cos(np.pi/2), r*np.sin(np.pi/2), 0) #peak of riblet
        vertices_bottom[5+i*n_bottom] = (r*np.cos(np.pi/4), r*np.sin(np.pi/4), 0) 
        vertices_bottom[6+i*n_bottom] = (r*np.cos(0), 0, 0) #base of riblet RHS
        vertices_bottom[7+i*n_bottom] = (R*np.cos(0), 0, 0) 
        vertices_bottom[8+i*n_bottom] = (W, 0, 0) #RHS
        
        vertices_middle[0+i*n_middle] = (-W, R*np.sin(3*np.pi/4), 0)
        vertices_middle[1+i*n_middle] = (R*np.cos(3*np.pi/4), R*np.sin(3*np.pi/4), 0)
        vertices_middle[2+i*n_middle] = (R*np.cos(np.pi/2), R*np.sin(np.pi/2), 0)
        vertices_middle[3+i*n_middle] = (R*np.cos(np.pi/4), R*np.sin(np.pi/4), 0)
        vertices_middle[4+i*n_middle] = (W, R*np.sin(np.pi/4), 0) 
        
        vertices_top[0+i*n_middle] = (-W, H, 0)
        vertices_top[1+i*n_middle] = (R*np.cos(3*np.pi/4), H, 0)
        vertices_top[2+i*n_middle] = (R*np.cos(np.pi/2), H, 0)
        vertices_top[3+i*n_middle] = (R*np.cos(np.pi/4), H, 0)
        vertices_top[4+i*n_middle] = (W, H, 0)
        
    vertices_bottom = [np.array(v) for v in vertices_bottom if v]    
    vertices_middle = [np.array(v) for v in vertices_middle if v]
    vertices_top = [np.array(v) for v in vertices_top if v]
        
    all_vertices = vertices_bottom
    all_vertices.extend(vertices_middle)
    all_vertices.extend(vertices_top)
    
    vertices_other = [v + np.array([0., 0., z]) for v in all_vertices] #add the other vertices in the z-direction
    all_vertices.extend(vertices_other)
    
    all_vertices_LoL = [list(a) for a in all_vertices] #creates list of lists
    all_vertices_LoLs = [['{:.5f}'.format(x) for x  in sublist] for sublist in all_vertices_LoL] #converts the sublists into strings    
    filtered_vertices = []
    for sublist in all_vertices_LoLs:
        if sublist not in filtered_vertices:
            filtered_vertices.append(sublist)
            
    filtered_vertices = [[float(x) for x in sublist] for sublist in filtered_vertices] #converts back to floats due to legacy formatting
    
    for i, elem in enumerate(filtered_vertices):
        print("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //vertex "+str(i))


In [8]:
#generateVertices(1, 1, 0.1, 5, 99)

In [ ]:
def generateHexBlocks(N):
    
    '''method to generate hex blocks for N prism riblets for use in openfoam.
    this version has not yet been updated to include the inlet/outlet blocks
    N = no. of riblets'''
    
    #the strings below are used to have their values read in from a file in the blockMeshDict folder
    ribletX_nC = '$ribletX_nC' #the no. of x-cells in the blocks on/above the riblets
    plateauX_nC = '$plateauX_nC' #the no. of x-cells in the blocks on/above the space between riblets
    surfaceY_nC = '$surfaceY_nC' #the no. of y-cells in the blocks on the surface
    fluidY_nC = '$fluidY_nC' #the no. of y-cells in the blocks in the surrounding fluid 
    
    LBlockX_nC = '$LBlockX_nC' #the no. of x-cells in the LHS inlet block
    RBlockX_nC = '$RBlockX_nC' #the no. of x-cells in the RHS outlet block
    
    nCells_z = '1' #only one cell in z-direction
    
    n_blocks = 10 #this is the number of blocks in each riblet subsystem.  Just used for keeping track of the indexing
    nb = 9N-(N-1) #no. of vertices on bottom row. nb-1 for the index of the last bottom vertex. nb is the index of the first mid vertex 
    nm = nb + (4*N + 1)
    nt = nm + (4*N + 1)